In [ ]:
!gdown 

In [ ]:
!gdown 
!unzip /content/CogentScanner.zip

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import csv
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device('/device:GPU:0')

Found GPU at: /device:GPU:0


In [ ]:
def mapped_labels(labels, colors, gender, black_white):
    labels_to_lens = {'Colored': 0, 'Normal': 1, 'Transparent': 2}
    lens_colors_types = {"Gray": 0, "Hazel": 1, "Green": 2, "Blue": 3}
    gender_types = {"Male": 0, "Female": 1}
    black_white_types = {"No": 0, "Yes": 1}


    labels = float(labels_to_lens[labels])
    gender = float(gender_types[gender])
    black_white = float(black_white_types[black_white])
    colors = float(lens_colors_types[colors])

    return labels, gender, black_white, colors

In [ ]:
def read_csv(train_test, number_of_itens = -1):
    data_labels = []
    imgs = []

    csvreader = None
    with open(f"/content/CogentAnnotation{train_test}.csv", "r") as file:
        csvreader = csv.reader(file)
        csvreader = list(csvreader)

    csvreader.pop(0)
    csvreader = np.array(csvreader)
    np.random.shuffle(csvreader)

    for row in csvreader:
        labels, gender, black_white, colors = mapped_labels(row[1], row[3], row[4], row[5])

        img_path = "/content/" + row[0].replace("&", "_") + ".bmp"
        img = img_to_array(load_img(img_path, target_size=(300,300))) / 255.
        imgs.append(img)

        data_labels.append([labels, colors, black_white, gender, row[6], row[7], row[8], row[9], row[10],row[11]])


    data_labels = (np.array(data_labels)).astype(np.float32)
    return data_labels.T, np.array(imgs)

treino_labels, treino_imgs = read_csv("Test")

In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten, BatchNormalization, Rescaling, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

#Modelo base
input = Input(shape=(300,300,3))
rescaling = Rescaling(scale=1./255)(input) #1
base_model = EfficientNetB3(input_shape=(300,300,3), include_top=False)
base_model.trainable = True
model = base_model(rescaling, training=True) #2
batch = BatchNormalization()(model) #3
globalPooling = GlobalAveragePooling2D()(batch) #4

#Camada densa 1
dropout = Dropout(0.145)(globalPooling)
dense1 = Dense(1536, activation='relu')(dropout)

#Camada densa 2
dropout1 = Dropout(0.1)(dense1)
dense2 = Dense(512, activation='relu')(dropout1)

#Predicao do modelo
prediction = Dense(3, activation='softmax')(dense2)


new_model = Model(input, prediction)
new_model.compile(optimizer=Adam(learning_rate=0.001), loss = SparseCategoricalCrossentropy(), metrics=['accuracy'])


43941136/43941136 [==============================] - 2s 0us/step


In [ ]:
history = new_model.fit(treino_imgs, treino_labels[0], epochs=10, batch_size=32, validation_split=0.3)

Epoch 1/10
39/39 [==============================] - 124s 1s/step - loss: 0.7496 - accuracy: 0.6034 - val_loss: 0.6814 - val_accuracy: 0.5939
Epoch 2/10
39/39 [==============================] - 41s 1s/step - loss: 0.5791 - accuracy: 0.6906 - val_loss: 0.6646 - val_accuracy: 0.6338
Epoch 3/10
39/39 [==============================] - 40s 1s/step - loss: 0.5621 - accuracy: 0.7028 - val_loss: 0.6179 - val_accuracy: 0.6812
Epoch 4/10
39/39 [==============================] - 41s 1s/step - loss: 0.4802 - accuracy: 0.7679 - val_loss: 0.5045 - val_accuracy: 0.7419
Epoch 5/10
39/39 [==============================] - 41s 1s/step - loss: 0.4035 - accuracy: 0.8331 - val_loss: 0.3799 - val_accuracy: 0.8235
Epoch 6/10
39/39 [==============================] - 41s 1s/step - loss: 0.2835 - accuracy: 0.8836 - val_loss: 0.4073 - val_accuracy: 0.8273
Epoch 7/10
39/39 [==============================] - 41s 1s/step - loss: 0.2364 - accuracy: 0.9104 - val_loss: 0.3712 - val_accuracy: 0.8425
Epoch 8/10
39/39 [=